In [21]:
import pandas as pd

df = pd.read_csv('dataset_clean.csv')
df.head()

,model,start_time,end_time,image_file,resolution_width,resolution_height,framework,fps,inference_time,cpu_usage,predicted_class,avg_voltage_V,avg_current_A,avg_temp_C,power_W
0,RESNET50,1.731266e+09,1.731266e+09,cat_0.jpg,224,224,PYTORCH,10.0,0.3499,96.88,898,5.146451,1.543984,30.100000,7.946038
1,RESNET50,1.731266e+09,1.731266e+09,cat_1.jpg,224,224,PYTORCH,10.0,0.3052,100.00,282,5.146517,1.612325,30.100000,8.297857
2,RESNET50,1.731266e+09,1.731266e+09,cat_2.jpg,224,224,PYTORCH,10.0,0.3046,100.00,283,5.143643,1.640716,30.153968,8.439259
3,RESNET50,1.731266e+09,1.731266e+09,dog_0.jpg,224,224,PYTORCH,10.0,0.3050,99.49,248,5.143671,1.667547,30.175129,8.577312
4,RESNET50,1.731266e+09,1.731266e+09,dog_1.jpg,224,224,PYTORCH,10.0,0.3584,100.00,233,5.143563,1.648180,30.200000,8.477518


In [22]:
# PyTorch
df = df[~((df['model'] == 'RESNET18') & (df['framework'] == 'PYTORCH') & (df['fps'] > 10))]
df = df[~((df['model'] == 'RESNET50') & (df['framework'] == 'PYTORCH') & (df['fps'] > 10))]
df = df[~((df['model'] == 'RESNEXT') & (df['framework'] == 'PYTORCH') & (df['fps'] > 10))]
df = df[~((df['model'] == 'MOBILENETV3') & (df['framework'] == 'PYTORCH') & (df['fps'] > 40))]
df = df[~((df['model'] == 'SQUEEZENET') & (df['framework'] == 'PYTORCH') & (df['fps'] > 30))]

# Hailo
df = df[~((df['model'] == 'RESNET18') & (df['framework'] == 'HAILO') & (df['fps'] > 320))]
df = df[~((df['model'] == 'RESNET50') & (df['framework'] == 'HAILO') & (df['fps'] > 50))]
df = df[~((df['model'] == 'RESNEXT') & (df['framework'] == 'HAILO') & (df['fps'] > 50))]
df = df[~((df['model'] == 'MOBILENETV3') & (df['framework'] == 'HAILO') & (df['fps'] > 140))]
df = df[~((df['model'] == 'SQUEEZENET') & (df['framework'] == 'HAILO') & (df['fps'] > 700))]

In [23]:
pytorch_df = df[df['framework'] == 'PYTORCH']
hailo_df = df[df['framework'] == 'HAILO']

In [24]:
import pandas as pd
import matplotlib.pyplot as plt

In [25]:
def aggregate_power_by_fps(data):
    # Group by fps and model
    grouped = data.groupby(['fps', 'model']).agg(
        mean_power=('power_W', 'mean')
    ).reset_index()
    return grouped

# Aggregate power consumption for PyTorch
pytorch_agg = aggregate_power_by_fps(pytorch_df)

# Aggregate power consumption for Hailo
hailo_agg = aggregate_power_by_fps(hailo_df)

In [26]:
# Aggregate power consumption for PyTorch
pytorch_agg = aggregate_power_by_fps(pytorch_df)

# Aggregate power consumption for Hailo
hailo_agg = aggregate_power_by_fps(hailo_df)

# Function to plot and save the graphs as PNG
def plot_and_save_power_vs_fps(agg_data, framework_name, fps_range, plot_title, file_name):
    plt.figure(figsize=(10, 6))

    # Filter the data for the given FPS range
    filtered_data = agg_data[(agg_data['fps'] >= fps_range[0]) & (agg_data['fps'] <= fps_range[1])]

    # Get the list of unique models
    models = filtered_data['model'].unique()

    # Plot each model
    for model in models:
        model_data = filtered_data[filtered_data['model'] == model]
        plt.plot(model_data['fps'], model_data['mean_power'], marker='o', label=model)

    plt.xlabel('FPS (Frames per Second)')
    plt.ylabel('Power Consumption (W)')
    plt.title(plot_title)
    plt.legend(title='Model')
    plt.grid(True)

    # Save the plot as a PNG file
    plt.savefig(file_name, format='png', dpi=300)  # 300 dpi for high resolution
    plt.close()  # Close the figure after saving

# Define FPS range from 10 to 700
fps_range_3 = (10, 700)

# Save PyTorch plots for FPS range 10 to 700
plot_and_save_power_vs_fps(pytorch_agg, 'PyTorch', fps_range_3, 'Power Consumption vs FPS (10 to 40) for PyTorch', 'pytorch_power_10_to_40.png')

# Save Hailo plots for FPS range 10 to 700
plot_and_save_power_vs_fps(hailo_agg, 'Hailo', fps_range_3, 'Power Consumption vs FPS (10 to 700) for Hailo', 'hailo_power_10_to_700.png')

In [27]:
def aggregate_cpu_usage_by_fps(data):
    # Group by fps and model
    grouped = data.groupby(['fps', 'model']).agg(
        mean_cpu_usage=('cpu_usage', 'mean')
    ).reset_index()
    return grouped

# Aggregate CPU usage for PyTorch
pytorch_cpu_agg = aggregate_cpu_usage_by_fps(pytorch_df)

# Aggregate CPU usage for Hailo
hailo_cpu_agg = aggregate_cpu_usage_by_fps(hailo_df)

# Function to plot and save the CPU usage vs FPS graphs as PNG
def plot_and_save_cpu_usage_vs_fps(agg_data, framework_name, fps_range, plot_title, file_name):
    plt.figure(figsize=(10, 6))

    # Filter the data for the given FPS range
    filtered_data = agg_data[(agg_data['fps'] >= fps_range[0]) & (agg_data['fps'] <= fps_range[1])]

    # Get the list of unique models
    models = filtered_data['model'].unique()

    # Plot each model
    for model in models:
        model_data = filtered_data[filtered_data['model'] == model]
        plt.plot(model_data['fps'], model_data['mean_cpu_usage'], marker='o', label=model)

    plt.xlabel('FPS (Frames per Second)')
    plt.ylabel('CPU Usage (%)')
    plt.title(plot_title)
    plt.legend(title='Model')
    plt.grid(True)

    # Save the plot as a PNG file
    plt.savefig(file_name, format='png', dpi=300)  # 300 dpi for high resolution
    plt.close()  # Close the figure after saving

# Define FPS range from 10 to 700
fps_range_3 = (10, 700)

# Save PyTorch CPU usage plots for FPS range 10 to 700
plot_and_save_cpu_usage_vs_fps(pytorch_cpu_agg, 'PyTorch', fps_range_3, 'CPU Usage vs FPS (10 to 40) for PyTorch', 'pytorch_cpu_10_to_40.png')

# Save Hailo CPU usage plots for FPS range 10 to 700
plot_and_save_cpu_usage_vs_fps(hailo_cpu_agg, 'Hailo', fps_range_3, 'CPU Usage vs FPS (10 to 700) for Hailo', 'hailo_cpu_10_to_700.png')


In [28]:
df.to_csv('dataset_clean.csv', index=False)